In [15]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from datetime import datetime, timedelta
from geopy.distance import geodesic
import random

random.seed(2002)


df = pd.read_csv('usuarios.csv', encoding='utf-8', index_col=0)
ids = np.unique(df['ID'])

df2 = pd.read_csv('trayectos_20_min.csv')
coord_ini = df2['Coordenadas_inicio']
coord_fin = df2['Coordenadas_fin']
tiempo = df2['Minutos']

In [16]:
# comenzamos a inventarnos datos

motivo = ['Estudios', 'Trabajo', 'Deporte', 'Ocio', 'Otro']
nombre_rutina_deporte = ['Futbol', 'Baloncesto', 'Yoga', 'Gym', 'Tenis', 'Padel', 'Baile']

disponibilidad = ['Coche', 'Pasajero', 'Coche y pasajero']
dias = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
ant_trayecto = ['Bus', 'Metro', 'Tren', 'Coche', 'Coche', 'Coche', 'Coche']
horas_salida = ['7:00']

while horas_salida[-1] != '21:00':
    hora_ultima = datetime.strptime(horas_salida[-1], '%H:%M')
    hora_next = hora_ultima + timedelta(minutes=5)
    hora_next = hora_next.strftime('%H:%M')
    horas_salida.append(hora_next)
    


nombre_rutina = []
id_usuario = []
lugar_salida = []
lugar_llegada = []
hora_salida = []
hora_llegada = []
motive = []
days = []
disp = []
viejo_trayecto = []



for i in range(50000):
    id_usuario.append(random.choice(ids))
    
    
    i = random.randint(0, 3499)
    lugar_salida.append(coord_ini[i][1:-1])
    lugar_llegada.append(coord_fin[i][1:-1])
    tiempo_min = tiempo[i]
    
    hora_sal = random.choice(horas_salida)
    hora_salida.append(hora_sal)
    hora_inicial_dt = datetime.strptime(hora_sal, '%H:%M')
    hora_final_dt = hora_inicial_dt + timedelta(minutes=tiempo_min)
    hora_final = hora_final_dt.strftime('%H:%M')
    hora_llegada.append(hora_final)

    
    m = random.choice(motivo)
    motive.append(m)
    
    if m == 'Estudios':
        d = random.choice(dias[:5])
        nom_rut = f'Estudios_universidad_{d.lower()}' # solo cogemos a que estudien en universidades
    elif m == 'Otro':
        d = random.choice(dias)
        nom_rut = f'[Indefinido]_{d.lower()}'
    elif m == 'Trabajo':
        d = random.choice(dias[:5])
        nom_rut = f'Trabajo_{d.lower()}'
    elif m == 'Ocio':
        d = random.choice(dias)
        nom_rut = f'Ocio_{d.lower()}'
    else:                                   # es deporte
        d = random.choice(dias)
        deport = random.choice(nombre_rutina_deporte)
        nom_rut = f'Deporte_{deport.lower()}_{d.lower()}'
    
    days.append(d)
    nombre_rutina.append(nom_rut)
    disp.append(random.choice(disponibilidad))
    viejo_trayecto.append(random.choice(ant_trayecto))



In [17]:
d = {'Nombre_rutina':nombre_rutina, 'Id_usuario': id_usuario, 'Lugar_salida':lugar_salida, 'Lugar_llegada':lugar_llegada, 
     'Hora_salida':hora_salida, 'Hora_llegada':hora_llegada, 'Motivo':motive, 'Dia':days, 'Disponibilidad_coche':disp,
     'Viejo_trayecto':viejo_trayecto}

df = pd.DataFrame(d)
df.head()

,Nombre_rutina,Id_usuario,Lugar_salida,Lugar_llegada,Hora_salida,Hora_llegada,Motivo,Dia,Disponibilidad_coche,Viejo_trayecto
0,[Indefinido]_lunes,111831,"39.50122800718866, -0.41927966292614793","39.44984907483212, -0.3923317810900036",13:50,14:12,Otro,Lunes,Pasajero,Tren
1,Estudios_universidad_miercoles,982437,"39.47448582694143, -0.36973136061629475","39.52827350270283, -0.3752537306405829",7:00,07:24,Estudios,Miercoles,Coche y pasajero,Tren
2,Deporte_futbol_miercoles,220786,"39.487288123748414, -0.3832181679770428","39.46320872584864, -0.378861657627463",14:15,14:25,Deporte,Miercoles,Pasajero,Bus
3,Deporte_tenis_viernes,362659,"39.464666255889306, -0.378869876395067","39.48122685723318, -0.357178565188734",15:25,15:35,Deporte,Viernes,Coche y pasajero,Coche
4,Ocio_viernes,693935,"39.41972386577771, -0.3892850451039514","39.45630189278762, -0.3795688513211948",07:50,08:09,Ocio,Viernes,Coche,Tren


In [18]:
# RUTINAS DUPLICADAS

dias = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']
filas_duplicadas = []
for dia in dias:
    for motivo in ['Estudios', 'Trabajo']:
        filas_dia_motivo = df[(df['Dia'] == dia) & (df['Motivo'] == motivo)].copy()
        if not filas_dia_motivo.empty:
            siguiente_dia = dias[(dias.index(dia) + 1) % len(dias)]
            filas_dia_motivo['Dia'] = siguiente_dia
            filas_dia_motivo['Nombre_rutina'] = motivo + '_' + siguiente_dia.lower()
            filas_duplicadas.append(filas_dia_motivo)

nuevo_df = pd.concat([df] + filas_duplicadas, ignore_index=True)

print(len(df), len(nuevo_df))
df = nuevo_df

50000 69979


In [19]:
df.tail()

,Nombre_rutina,Id_usuario,Lugar_salida,Lugar_llegada,Hora_salida,Hora_llegada,Motivo,Dia,Disponibilidad_coche,Viejo_trayecto
69974,Trabajo_lunes,816759,"39.492830843796014, -0.39243040931334794","39.45995740816682, -0.37352463166617994",09:35,09:50,Trabajo,Lunes,Coche y pasajero,Coche
69975,Trabajo_lunes,258511,"39.513050653598384, -0.41248790485508674","39.50307629454208, -0.4441124080585561",09:40,09:56,Trabajo,Lunes,Coche y pasajero,Coche
69976,Trabajo_lunes,772650,"39.456873805434554, -0.3934430774184928","39.464281735199975, -0.34041612598277016",14:40,14:57,Trabajo,Lunes,Coche,Coche
69977,Trabajo_lunes,276077,"39.44089067251417, -0.4055396401894798","39.45420482476331, -0.3940216832851793",07:15,07:26,Trabajo,Lunes,Coche,Coche
69978,Trabajo_lunes,415433,"39.469784606924335, -0.3877803534861608","39.492411749362866, -0.37997700453719324",13:45,13:56,Trabajo,Lunes,Pasajero,Bus


In [20]:
# NUEVAS FILAS DE ESTUDIOS Y TRABAJO

filas_necesarias = 70000 - 10 - len(df) 

# Crear una lista de Motivos alternativos (Estudios o Trabajo)
motivos_alternativos = ['Estudios', 'Trabajo']

# Seleccionar las filas cuyo Motivo es 'Ocio'
filas_ocio = df[df['Motivo'] == 'Ocio']

# Si hay más filas de 'Ocio' de las necesarias, toma solo las primeras
if len(filas_ocio) > filas_necesarias:
    filas_ocio = filas_ocio.iloc[:filas_necesarias]

# Cambiar el Motivo a 'Estudios' o 'Trabajo' de manera aleatoria
filas_ocio['Motivo'] = np.random.choice(motivos_alternativos, size=len(filas_ocio))

# Cambiar el Día a un día de semana si es Sábado o Domingo
dias_semana = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']
filas_ocio['Dia'] = np.where(filas_ocio['Dia'].isin(['Sabado', 'Domingo']), np.random.choice(dias_semana, size=len(filas_ocio)), filas_ocio['Dia'])
filas_ocio['Nombre_rutina'] = filas_ocio['Motivo'] + '_' + filas_ocio['Dia'].str.lower()


# Agregar las filas modificadas al DataFrame original
df = pd.concat([df, filas_ocio], ignore_index=True)

In [21]:
df.tail()

,Nombre_rutina,Id_usuario,Lugar_salida,Lugar_llegada,Hora_salida,Hora_llegada,Motivo,Dia,Disponibilidad_coche,Viejo_trayecto
69985,Trabajo_martes,125796,"39.47939649928545, -0.3771206771992797","39.499527474914395, -0.42242998224472217",15:15,15:34,Trabajo,Martes,Coche y pasajero,Coche
69986,Estudios_viernes,359492,"39.50971200414956, -0.44623920702002756","39.51481879478544, -0.41535239935653245",14:40,14:53,Estudios,Viernes,Coche,Coche
69987,Trabajo_viernes,776562,"39.50293637495475, -0.35025506429004816","39.449733140747554, -0.3969585182627564",07:30,08:08,Trabajo,Viernes,Pasajero,Coche
69988,Trabajo_lunes,275817,"39.47883390886615, -0.3720843014131642","39.44834644250786, -0.33141219772190644",08:45,09:04,Trabajo,Lunes,Pasajero,Coche
69989,Estudios_lunes,707278,"39.52052760344113, -0.387615329474106","39.48387276527541, -0.39536020180169285",08:10,08:26,Estudios,Lunes,Pasajero,Coche


In [22]:
# DATOS DE PRUEBA (TODOS DEBERÍAN SER MATCHING DE prueba_1)

nueva_fila = {'Nombre_rutina':'prueba_1', 'Id_usuario':'913690', 'Lugar_salida':["39.47679696739808, -0.3462473060000242"], 
              'Lugar_llegada':["39.405576, -0.406062"], 'Hora_salida':'7:30', 'Hora_llegada':'8:00', 'Motivo':'Estudios',
              'Dia':'Lunes', 'Disponibilidad_coche':'Coche y pasajero', 'Viejo_trayecto':'Coche'}

# primera prueba
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con un lugar de salida cercano
nueva_fila['Id_usuario'] = '156939'
nueva_fila['Nombre_rutina'] = 'prueba_2'
nueva_fila['Lugar_salida'] = ["39.478350, -0.347192"]
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con un lugar de llegada cercano
nueva_fila['Nombre_rutina'] = 'prueba_3'
nueva_fila['Lugar_llegada'] = ["39.404948, -0.406213"]
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con un motivo distinto
nueva_fila['Nombre_rutina'] = 'prueba_4'
nueva_fila['Motivo'] = "Trabajo"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una hora de salida distinta
nueva_fila['Nombre_rutina'] = 'prueba_5'
nueva_fila['Hora_salida'] = "7:20"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una hora de salida distinta
nueva_fila['Nombre_rutina'] = 'prueba_6'
nueva_fila['Hora_salida'] = "7:25"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una hora de llegada distinta
nueva_fila['Nombre_rutina'] = 'prueba_7'
nueva_fila['Hora_llegada'] = "7:50"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una hora de llegada distinta
nueva_fila['Nombre_rutina'] = 'prueba_8'
nueva_fila['Hora_llegada'] = "7:55"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una disponibilidad distinta
nueva_fila['Nombre_rutina'] = 'prueba_9'
nueva_fila['Disponibilidad_coche'] = "Coche"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una disponibilidad distinta
nueva_fila['Nombre_rutina'] = 'prueba_10'
nueva_fila['Disponibilidad_coche'] = "Pasajero"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)


In [23]:
df.tail()

,Nombre_rutina,Id_usuario,Lugar_salida,Lugar_llegada,Hora_salida,Hora_llegada,Motivo,Dia,Disponibilidad_coche,Viejo_trayecto
69995,prueba_6,156939,"39.478350, -0.347192","39.404948, -0.406213",7:25,8:00,Trabajo,Lunes,Coche y pasajero,Coche
69996,prueba_7,156939,"39.478350, -0.347192","39.404948, -0.406213",7:25,7:50,Trabajo,Lunes,Coche y pasajero,Coche
69997,prueba_8,156939,"39.478350, -0.347192","39.404948, -0.406213",7:25,7:55,Trabajo,Lunes,Coche y pasajero,Coche
69998,prueba_9,156939,"39.478350, -0.347192","39.404948, -0.406213",7:25,7:55,Trabajo,Lunes,Coche,Coche
69999,prueba_10,156939,"39.478350, -0.347192","39.404948, -0.406213",7:25,7:55,Trabajo,Lunes,Pasajero,Coche


In [24]:
df = df.drop_duplicates()
df.to_csv("trayectos.csv",index_label=None)